In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from weather.noaa import load_isd_history, load_isd
import os
from os.path import join
import gridemissions
from gridemissions.viz import set_plots

In [ ]:
from gridemissions.papers.physics_informed import get_changes

In [ ]:
print(gridemissions.config["APEN_PATH"])
FIG_PATH = join(gridemissions.config["APEN_PATH"], "figures")
os.makedirs(FIG_PATH, exist_ok=True)

In [ ]:
COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()

In [ ]:
df_stations = load_isd_history(subset=None)

In [ ]:
# name_dict = df_stations.loc[df_stations.fileNm.isin(["725180-14735", "999999-54902", "722540-13904"]), :][["fileNm", "STATION NAME"]].set_index("fileNm")["STATION NAME"].to_dict()
name_dict = {
    "722540-13904": "Austin airport",
    "725180-14735": "Albany airport",
    "999999-54902": "Des Moines",
}

In [ ]:
df = load_isd(
    file_name=["725180-14735", "999999-54902", "722540-13904"],
    years=range(2015, 2021),
    request_type="list",
)

In [ ]:
def make_plot(ax, s, color, label, alpha=1.0, lw=1.0):
    xvals = pd.to_datetime(
        (10000 * 2020 + 100 * s.index.month + s.index.day).astype(str)
    )
    ax.plot(xvals, s, label=label, color=color, alpha=alpha, lw=lw)


f, (ax, ax2) = plt.subplots(
    2,
    3,
    figsize=(1.5 * PAGE_WIDTH, 1.2 * ROW_HEIGHT),
    gridspec_kw={"height_ratios": [2.7, 1]},
)

for i, k in enumerate(df.keys()):
    df_plot = df[k].TEMP.groupby(df[k].index.date).mean().rolling(14).mean()
    df_plot.index = pd.to_datetime(df_plot.index)
    df_plot = df_plot["20150714":]

    for iyear, year in enumerate([2015, 2016, 2017, 2018, 2019, 2020]):
        if year == 2020:
            color = "k"
            alpha = 1
            lw = 1
        else:
            color = COLORS[iyear]
            alpha = 0.6
            lw = 0.5
        make_plot(
            ax[i],
            df_plot.loc[df_plot.index.year == year],
            color=color,
            label=year,
            alpha=alpha,
            lw=lw,
        )
        ax[i].set_title(name_dict[k])

    changes = get_changes(df_plot, which="abs")
    ax2[i].plot(changes)

import matplotlib.dates as mdates

for a in ax:
    a.set_xticklabels([])
    a.tick_params(length=0.0)
    a.set_ylabel("F")
for a in ax2:
    a.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    for label in a.get_xticklabels():
        label.set_ha("right")
        label.set_rotation(20)
    a.set_ylabel("F")
ax[1].legend(loc=3, ncol=2)
f.suptitle("(d) Mean daily temperature", ha="left", x=0.01, y=0.99)
f.subplots_adjust(left=0.06, bottom=0.2, right=0.99, top=0.84, wspace=0.3, hspace=0.08)
f.savefig(join(FIG_PATH, "fig7_temps.pdf"))